In [1]:
!pip install SpeechRecognition
!pip install gTTS
!pip install playsound3
!pip install pyjokes
!pip install wikipedia
!pip install pygame
!pip install PyAudio
!pip install sounddevice

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 18.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=98b2f17cc53eca3befc3484961be17d648007addda28745582f4da6c32f60a47
  Stored in directory: /home/carlos/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyAudio: filename=PyAudio-0.2.14-cp311-cp311-linux_x86_64.whl size=27346 sha256=5c2c4857c21bc2d9eb555ab47e279a2e50b8715b265a900afa3ee0103b232629
  Stored in directory: /home/carlos/.cache/pip/wheels/80/b1/c1/67e4ef443de2665d86031d4760508094eab5de37d5d64d9c27
Successfully built PyAudio


In [3]:
import speech_recognition as sr

def speech_to_text_and_save():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Say something:")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source, timeout=5)

    try:
        print("Recognizing...")
        text = recognizer.recognize_google(audio)
        print(f"Text: {text}")

        with open("transcribed_text.txt", "w") as file:
            file.write(text)
            print("Transcribed text saved to transcribed_text.txt")
    except sr.UnknownValueError:
        print("Could not understand audio.")
    except sr.RequestError as e:
        print(f"Error connecting to Google API: {e}")

if __name__ == "__main__":
    speech_to_text_and_save()

ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib dlmisc.c:339:(snd_dlobj_cache_get0) Cannot open shared library libasound_module_pcm_pipewire.so (/home/carlos/anaconda3/envs/pytorch/lib/alsa-lib/libasound_module_pcm_pipewire.so: cannot open shared object file: No such file or directory)
ALSA lib dlmisc.c:339:(snd_dlobj_cache_get0) Cannot open shared library libasound_module_pcm_pipewire.so (/home/carlos/anaconda3/envs/pytorch/lib/alsa-lib/libasound_module_pcm_pipewire.so: cannot open shared object file: No such file or directory)
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib dlmisc.c:339:(snd_dlobj_cache_get0) Cannot open shared library libasound_mo

Say something:


WaitTimeoutError: listening timed out while waiting for phrase to start

In [5]:
# Importação de bibliotecas
import speech_recognition as sr
from gtts import gTTS
import os
from datetime import datetime
import sounddevice as sd
import numpy as np
import webbrowser
import pyjokes
import wikipedia
from pygame import mixer
import wave

# Captura de áudio usando sounddevice
def get_audio():
    fs = 44100  # Taxa de amostragem
    seconds = 5  # Duração da gravação

    print("Estou ouvindo...")
    recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1, dtype=np.int16)
    sd.wait()

    filename = "audio.wav"
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(fs)
        wf.writeframes(recording.tobytes())

    r = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = r.record(source)
        said = ""
        try:
            said = r.recognize_google(audio)
            print(said)
        except sr.UnknownValueError:
            speak("Desculpe, não entendi.")
        except sr.RequestError:
            speak("O serviço não está disponível.")

    return said.lower()

# Função para converter texto em fala
def speak(text):
    tts = gTTS(text=text, lang='en')
    filename = "voice.mp3"
    try:
        os.remove(filename)
    except OSError:
        pass
    tts.save(filename)
    mixer.init()
    mixer.music.load(filename)
    mixer.music.play()

# Função para responder comandos
def respond(text):
    print("Texto reconhecido: " + text)
    if 'youtube' in text:
        speak("O que você quer pesquisar?")
        keyword = get_audio()
        if keyword:
            url = f"https://www.youtube.com/results?search_query={keyword}"
            webbrowser.open(url)
            speak(f"Aqui estão os resultados para {keyword} no YouTube")
    elif 'search' in text:
        speak("O que você quer pesquisar?")
        query = get_audio()
        if query:
            result = wikipedia.summary(query, sentences=3)
            speak("De acordo com a Wikipédia")
            print(result)
            speak(result)
    elif 'joke' in text:
        speak(pyjokes.get_joke())
    elif 'what time' in text:
        strTime = datetime.today().strftime("%H:%M %p")
        print(strTime)
        speak(strTime)
    elif 'play music' in text or 'play song' in text:
        speak("Reproduzindo música...")
        music_dir = "/home/carloshenrique/Documents/Git/dataExplorationAndML/DIO_Course/speechRecognizer"  # Modifique para seu diretório de músicas
        songs = os.listdir(music_dir)
        print(songs)
        playmusic(music_dir + "\\" + songs[0])
    elif 'stop music' in text:
        speak("Parando música.")
        stopmusic()
    elif 'exit' in text:
        speak("Até a próxima!")
        exit()

# Função para tocar música
def playmusic(song):
    mixer.init()
    mixer.music.load(song)
    mixer.music.play()

# Função para parar música
def stopmusic():
    mixer.music.stop()

# Execução principal
text = get_audio()
if text:
    respond(text)


Estou ouvindo...
